 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

This is simply a notebook to help me test stuff for the preprocessing script.

In [35]:
import netCDF4 as nc
import os

In [ ]:
os.path.join()

In [25]:
data_dir = '/Volumes/STICK/data/SPCAM/Aquaplanet/'
%ls $data_dir

AndKua_aqua_SPCAM3.0.cam2.h1.0000-01-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-02-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-03-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-04-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-06-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-09-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0000-12-01-00000.nc*
AndKua_aqua_SPCAM3.0.cam2.h1.0001-01-01-00000.nc*


In [26]:
rg = nc.Dataset(data_dir + 'AndKua_aqua_SPCAM3.0.cam2.h1.0000-01-01-00000.nc')
rg

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    source: CAM
    case: AndKua_aqua_SPCAM3.0
    title: 
    logname: tg802402
    host: c532-003.stamped
    Version: $Name:  $
    revision_Id: $Id: history.F90,v 1.26.2.38 2003/12/15 18:52:35 hender Exp $
    dimensions(sizes): lat(64), lon(128), lev(30), ilev(31), isccp_prs(7), isccp_tau(7), isccp_prstau(49), time(48), tbnd(2), chars(8), crm_x(32), crm_y(1), crm_z(28)
    variables(dimensions): float64 P0(), float64 lat(lat), float64 lon(lon), float64 crm_x(crm_x), float64 crm_y(crm_y), float64 crm_z(crm_z), float64 lev(lev), float64 ilev(ilev), float64 isccp_prs(isccp_prs), float64 isccp_tau(isccp_ta), float64 isccp_prstau(isccp_prsta), float64 time(time), float64 time_bnds(time,tbnd), |S1 date_written(time,chars), |S1 time_written(time,chars), int32 ntrm(), int32 ntrn(), int32 ntrk(), int32 ndbase(), int32 nsbase(), int32 nbdate(), int32 nbsec(), int32 mdt(),

In [27]:
time = rg.variables['time']
time

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: days since 0000-01-01 00:00:00
    calendar: noleap
    bounds: time_bnds
unlimited dimensions: time
current shape = (48,)
filling off

In [28]:
time[:]

array([ 0.        ,  0.02083333,  0.04166667,  0.0625    ,  0.08333333,
        0.10416667,  0.125     ,  0.14583333,  0.16666667,  0.1875    ,
        0.20833333,  0.22916667,  0.25      ,  0.27083333,  0.29166667,
        0.3125    ,  0.33333333,  0.35416667,  0.375     ,  0.39583333,
        0.41666667,  0.4375    ,  0.45833333,  0.47916667,  0.5       ,
        0.52083333,  0.54166667,  0.5625    ,  0.58333333,  0.60416667,
        0.625     ,  0.64583333,  0.66666667,  0.6875    ,  0.70833333,
        0.72916667,  0.75      ,  0.77083333,  0.79166667,  0.8125    ,
        0.83333333,  0.85416667,  0.875     ,  0.89583333,  0.91666667,
        0.9375    ,  0.95833333,  0.97916667])

In [46]:
lats = rg.variables['lat'][:]

In [ ]:
[]

In [42]:
tap = rg.variables['TAP']

In [44]:
tap.dimensions

('time', 'lev', 'lat', 'lon')

In [9]:
nc.Dataset(data_dir + 'AndKua_aqua_SPCAM3.0.cam2.h1.0001-01-01-00000.nc').variables['time'][:]

array([ 365.        ,  365.02083333,  365.04166667,  365.0625    ,
        365.08333333,  365.10416667,  365.125     ,  365.14583333,
        365.16666667,  365.1875    ,  365.20833333,  365.22916667,
        365.25      ,  365.27083333,  365.29166667,  365.3125    ,
        365.33333333,  365.35416667,  365.375     ,  365.39583333,
        365.41666667,  365.4375    ,  365.45833333,  365.47916667,
        365.5       ,  365.52083333,  365.54166667,  365.5625    ,
        365.58333333,  365.60416667,  365.625     ,  365.64583333,
        365.66666667,  365.6875    ,  365.70833333,  365.72916667,
        365.75      ,  365.77083333,  365.79166667,  365.8125    ,
        365.83333333,  365.85416667,  365.875     ,  365.89583333,
        365.91666667,  365.9375    ,  365.95833333,  365.97916667])

In [10]:
nc.Dataset('/Volumes/STICK/data/SPCAM/sample/SPCAM_outputs.nc')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): t(100000), z(21)
    variables(dimensions): float64 LAT(t), float64 PS(t), float64 SHFLX(t), float64 LHFLX(t), float64 OMEGA(z,t), float64 QAP(z,t), float64 TAP(z,t), float64 QRL(z,t), float64 QRS(z,t), float64 UBSP(z,t), float64 VBSP(z,t), float64 SPQC(z,t), float64 SPQG(z,t), float64 SPQI(z,t), float64 SPQR(z,t), float64 SPMC(z,t), float64 SPMCDN(z,t), float64 SPMCUDN(z,t), float64 SPMCUP(z,t), float64 SPMCUUP(z,t), float64 TPHYSTND(z,t), float64 PHQ(z,t), float64 SPDT(z,t), float64 SPDQ(z,t), float64 SPDQC(z,t), float64 SPDQI(z,t), float64 dTdt_adiabatic(z,t), float64 dQdt_adiabatic(z,t), float64 GRAD_UQ_H(z,t)
    groups: 

In [13]:
64 * 128 * 48

393216

In [16]:
4.5e6 / (64 * 128 * 48)

11.444091796875